In [4]:
# Forecast Evaluation Metrics
# ---------------------------
# This file includes the mathematical formulation and Python + MATLAB implementation
# for four key metrics used to compare forecasted sequences against actual values.

# ---------------------------
# Python Implementations
# ---------------------------

import numpy as np
import pandas as pd
import os
from typing import List
from dtaidistance import dtw
from properscoring import crps_ensemble
from dtaidistance import dtw


# 1. MAE - Mean Absolute Error
def mean_absolute_error(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return np.mean(np.abs(y_true - y_pred))

def mae_std(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return np.std(np.abs(y_true - y_pred))

# 2. DTW - Dynamic Time Warping Distance
def dtw_distance(y_true: List[float], y_pred: List[float]) -> float:
    return dtw.distance(y_true, y_pred)

# 3. Rolling RMSE
def rolling_rmse(y_true: np.ndarray, y_pred: np.ndarray, window: int = 5) -> np.ndarray:
    errors = (y_true - y_pred)**2
    rolling = np.convolve(errors, np.ones(window)/window, mode='valid')
    return np.sqrt(rolling)

def rolling_rmse_stats(y_true: np.ndarray, y_pred: np.ndarray, window: int = 5):
    rmse_vals = []
    for i in range(y_true.shape[0]):
        rr = rolling_rmse(y_true[i], y_pred[i], window)
        rmse_vals.append(rr)
    rmse_vals = np.concatenate(rmse_vals)
    return np.mean(rmse_vals), np.std(rmse_vals)

# 4. CRPS - Continuous Ranked Probability Score
def crps_score(y_true: np.ndarray, y_samples: np.ndarray) -> float:
    return np.mean(crps_ensemble(y_true, y_samples))

def crps_std(y_true: np.ndarray, y_samples: np.ndarray) -> float:
    return np.std(crps_ensemble(y_true, y_samples))


In [33]:
def evaluate_forecast(Y_TEST, model_name):
    results = []
    for trial in range(0, 49):
        test_path = f"/home/chiara/Energy/PRED/{model_name}_predictions_{trial}.csv"
        Y_PREDICT = pd.read_csv(test_path, header=0, sep=',', index_col="index")
        print(f'load: {test_path}')
        #print(f'The predicted test set (trial {trial}) has size of: {Y_PREDICT.shape}')
        for row in range(1, 127):
            if row >= len(Y_PREDICT.index):
                continue

            split_index = Y_PREDICT.index[row].split("_")
            if len(split_index) <= 1:
                continue

            key = split_index[3]
            pred_filtered_rows = Y_PREDICT[Y_PREDICT.index.str.contains(f"Ausgrid_Load_User_{key}_")].to_numpy()
            y_pred = np.concatenate(pred_filtered_rows, axis=0)
            true_filtered_rows = Y_TEST[Y_TEST.index.str.contains(f"Ausgrid_Load_User_{key}_")].to_numpy()
            y_true = np.concatenate(true_filtered_rows, axis=0)

            mae = mean_absolute_error(y_true, y_pred)
            mae_s = mae_std(y_true, y_pred)

            dtw_dist = dtw.distance(y_true, y_pred)
            rmse_m, rmse_s = rolling_rmse_stats(y_true, y_pred)
            crps = crps_score(y_true, y_pred)
            crps_s = crps_std(y_true, y_pred)
            results.append({
                "trial": trial,
                "key": key,
                "MAE": mae,
                "MAE_std": mae_s,
                "DTW": dtw_dist,
                "Rolling_RMSE": rmse_m,
                "Rolling_RMSE_std": rmse_s,
                "CRPS": crps,
                "CRPS_std": crps_s
            })

    return pd.DataFrame(results)


for model_name in ["gan", "VAE"]:
    test_path = "/home/chiara/Energy/Data/Test_2m_72.csv"
    Y_TEST = pd.read_csv(test_path, header=0, sep=';', decimal=",", index_col="index")
    Y_TEST = Y_TEST[[col for col in Y_TEST.columns if 'y' in col]]    res_df = evaluate_forecast(Y_TEST, model_name)
    #print('mean' ,res_df.mean(axis=0))
    


load: /home/chiara/Energy/PRED/gan_predictions_0.csv


KeyboardInterrupt: 

In [31]:
res_df.select_dtypes(include=[np.number]).mean()

trial               24.000000
MAE                  0.374135
MAE_std              0.463662
DTW                  5.169534
Rolling_RMSE         0.167318
Rolling_RMSE_std     0.207356
CRPS                 0.374135
CRPS_std             0.463662
dtype: float64

In [32]:
print(res_df.groupby("trial").mean(numeric_only=True).mean())  # Mean of metrics across all trials
print(res_df.groupby("key").mean(numeric_only=True))           # Or per key if useful


MAE                 0.374135
MAE_std             0.463662
DTW                 5.169534
Rolling_RMSE        0.167318
Rolling_RMSE_std    0.207356
CRPS                0.374135
CRPS_std            0.463662
dtype: float64
     trial       MAE   MAE_std       DTW  Rolling_RMSE  Rolling_RMSE_std  \
key                                                                        
1     24.0  0.382239  0.456430  5.060755      0.170942          0.204122   
10    24.0  0.385166  0.462975  4.840987      0.172252          0.207049   
11    24.0  0.450272  0.484126  4.523555      0.201368          0.216508   
12    24.0  0.351857  0.475033  4.945252      0.157355          0.212441   
13    24.0  0.400831  0.418988  4.685961      0.179257          0.187377   
14    24.0  0.421967  0.454923  4.456769      0.188710          0.203448   
15    24.0  0.389791  0.460608  4.609366      0.174320          0.205990   
16    24.0  0.365953  0.465997  5.057696      0.163659          0.208400   
17    24.0  0.323172  